In [ ]:
!pip install datatable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 7.2 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import datatable as dt

In [ ]:
#function for FAERS data processing
def FAERS_data_processing(time_period):

  #read in all the data files
  demo_data = pd.read_csv('DEMO'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
  drug_data = pd.read_csv('DRUG'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
  indi_data = pd.read_csv('INDI'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
  outc_data = pd.read_csv('OUTC'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
  #reac_data = pd.read_csv('REAC'+time_period+'.txt', delimiter='$', header=0) don't consider for now
  rpsr_data = pd.read_csv('RPSR'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
  #ther_data = pd.read_csv('THER'+time_period+'.txt', delimiter='$', header=0) no relevant columns

  #do pre-processing for the data file

  #for demo_data, remove any duplicates following FDA's logic
  #sort demo_data by primaryid and caseid and case version: reference: https://github.com/kylechua/faers-toolkit
  demo_data = demo_data.sort_values(by=['caseid','caseversion'])
  #keep only the latest case version
  demo_data = demo_data.drop_duplicates(subset=['caseid'], keep='last')
  #drop the following columns
  demo_data = demo_data.drop(['event_dt','mfr_dt', 'fda_dt','mfr_num','wt','wt_cod','rept_dt','occr_country','to_mfr','occp_cod'], axis=1)

  #remove redundant values from drug_data
  drug_data = drug_data[['primaryid','caseid', 'drug_seq','role_cod','drugname']]

  #deduplicat outc_data
  outc_data=outc_data.drop_duplicates()
  #for outc_table, combine all outcome codes into single row
  #outc_data = outc_data.groupby(['caseid'])['outc_cod'].apply('+'.join).reset_index()

  #now join the tables
  #join demo_data and drug_data and primaryid and caseid
  temp_1 = pd.merge(demo_data, drug_data, on=['primaryid','caseid'], how='left')
  #join temp1 with indi_data
  temp_2 = pd.merge(temp_1, indi_data,  left_on=['primaryid','caseid','drug_seq'], right_on=['primaryid','caseid','indi_drug_seq'], how='left')
  #join with outc_data
  temp_3 = pd.merge(temp_2, outc_data, on=['primaryid','caseid'], how='left')
  #now join with rpsr_data
  temp_4 = pd.merge(temp_3, rpsr_data, on=['primaryid','caseid'], how='left')

  del demo_data
  del drug_data
  del indi_data
  del outc_data
  del rpsr_data
  del temp_1
  del temp_2
  del temp_3

  #add year
  temp_4['year'] = pd.to_datetime(temp_4['init_fda_dt'], format='%Y%m%d').dt.year

  #categorize ages into groups
  #first standardize the age based on age_cod
  temp_4['final_age'] = np.select(
        [
            temp_4['age_cod'] == 'YR',
            temp_4['age_cod'] == 'MON',
            temp_4['age_cod'] == 'WK',
            temp_4['age_cod'] == 'DY',
            temp_4['age_cod'] == 'HR',
            temp_4['age_cod'] == 'DEC'
        ],
        [
            temp_4['age'] * 1,
            temp_4['age'] * (1/12),
            temp_4['age'] * (1/52),
            temp_4['age'] * (1/365),
            temp_4['age'] / (365*24),
            temp_4['age'] * 10
        ],
        default=1  # Handle missing or invalid ages if needed
    )

  #group the age into categories
  # Now categorize the 'final_age' column into age groups using pd.cut
  bins = [0, 1, 5, 12, 18, 65, 120]  # Define age group boundaries
  labels = ['Infant', 'Toddler', 'Child', 'Adolescent', 'Adult', 'Elderly']
  temp_4['age_category'] = pd.cut(temp_4['final_age'], bins=bins, labels=labels, include_lowest=True, right=False)

  ###final results aggregation
  #case_counts = temp_4.groupby(['drugname','prod_ai','mfr_sndr','role_cod','rept_cod', 'year','age_category','sex','indi_pt','outc_cod'])['caseid'].nunique().reset_index()
  #case_counts.rename(columns={'caseid': 'distinct_case_count'}, inplace=True)
  dt_df = dt.Frame(temp_4)
  grouped = dt_df[:, dt.nunique(dt.f.caseid), dt.by(dt.f.drugname, #dt.f.prod_ai,
                                                    dt.f.role_cod, dt.f.year, dt.f.age_category #dt.f.sex
                                                    , dt.f.indi_pt, dt.f.outc_cod)]
  # Print or display the results
  result_df = grouped.to_pandas()

  pivot_df = pd.pivot_table(
    result_df,
    values='caseid',  # Column to aggregate (replace with your count column)
    index=['drugname', #'prod_ai',
           'year', 'age_category', #'sex',
           'indi_pt', 'outc_cod'],  # Columns to keep as rows
    columns=['role_cod'],  # Column to pivot as columns
    aggfunc='sum',  # Aggregation function (replace if needed)
    fill_value=0  # Fill missing values with 0
  )

  pivot_df.columns = [f'{col}' for col in pivot_df.columns] #Flatten MultiIndex columns
  pivot_df = pivot_df.reset_index() #reset index

  return(pivot_df)



In [ ]:
#output_25Q1 = FAERS_data_processing('25Q1')
#output_24Q4 = FAERS_data_processing('24Q4')
#output_24Q3 = FAERS_data_processing('24Q3')
#output_24Q2 = FAERS_data_processing('24Q2')
#output_24Q1 = FAERS_data_processing('24Q1')
#output_23Q4 = FAERS_data_processing('23Q4')
#output_23Q3 = FAERS_data_processing('23Q3')
#output_23Q2 = FAERS_data_processing('23Q2')
#output_23Q1 = FAERS_data_processing('23Q1')
#output_22Q4 = FAERS_data_processing('22Q4')
#output_22Q3 = FAERS_data_processing('22Q3')
#output_22Q2 = FAERS_data_processing('22Q2')
#output_22Q1 = FAERS_data_processing('22Q1')
#output_21Q4 = FAERS_data_processing('21Q4')
#output_21Q3 = FAERS_data_processing('21Q3')
#output_21Q2 = FAERS_data_processing('21Q2')
#output_21Q1 = FAERS_data_processing('21Q1')
#output_20Q1 = FAERS_data_processing('20Q1')
#output_20Q2 = FAERS_data_processing('20Q2')
#output_20Q3 = FAERS_data_processing('20Q3')
#output_20Q4 = FAERS_data_processing('20Q4')
#output_19Q4 = FAERS_data_processing('19Q4')
#output_19Q3 = FAERS_data_processing('19Q3')
#output_19Q2 = FAERS_data_processing('19Q2')
#output_19Q1 = FAERS_data_processing('19Q1')
#output_18Q1 = FAERS_data_processing('18Q1')
#output_18Q2 = FAERS_data_processing('18Q2')
#output_18Q3 = FAERS_data_processing('18Q3')
#output_18Q4 = FAERS_data_processing('18Q4')
#output_17Q1 = FAERS_data_processing('17Q1')
#output_17Q2 = FAERS_data_processing('17Q2')
#output_17Q3 = FAERS_data_processing('17Q3')
#output_17Q4 = FAERS_data_processing('17Q4')
#output_16Q1 = FAERS_data_processing('16Q1')
#output_16Q2 = FAERS_data_processing('16Q2')
#output_16Q3 = FAERS_data_processing('16Q3')
#output_16Q4 = FAERS_data_processing('16Q4')
#output_15Q1 = FAERS_data_processing('15Q1')
#output_15Q2 = FAERS_data_processing('15Q2')
#output_15Q3 = FAERS_data_processing('15Q3')
#output_15Q4 = FAERS_data_processing('15Q4')
output_14Q1 = FAERS_data_processing('14Q1')
output_14Q2 = FAERS_data_processing('14Q2')
output_14Q3 = FAERS_data_processing('14Q3')
output_14Q4 = FAERS_data_processing('14Q4')

<ipython-input-56-6a65316ec3f9>:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  demo_data = pd.read_csv('DEMO'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
<ipython-input-56-6a65316ec3f9>:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  drug_data = pd.read_csv('DRUG'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
<ipython-input-56-6a65316ec3f9>:6: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  drug_data = pd.read_csv('DRUG'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
<ipython-input-56-6a65316ec3f9>:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  demo_data = pd.read_csv('DEMO'+time_period+'.txt', delimiter='$', header=0, encoding='latin-1')
<ipython-input-56-6a65316ec3f9>:6: DtypeWarning: Columns (10,

In [ ]:
#save the compelted dataframes to the drive
#output_25Q1.to_csv('output_25Q1.csv', index=False)
#output_24Q4.to_csv('output_24Q4.csv', index=False)
#output_24Q3.to_csv('output_24Q3.csv', index=False)
#output_24Q2.to_csv('output_24Q2.csv', index=False)
#output_24Q1.to_csv('output_24Q1.csv', index=False)
#output_23Q4.to_csv('output_23Q4.csv', index=False)
#output_23Q3.to_csv('output_23Q3.csv', index=False)
#output_23Q2.to_csv('output_23Q2.csv', index=False)
#output_23Q1.to_csv('output_23Q1.csv', index=False)

In [ ]:
#append dataframes together
output_2014 = pd.concat([output_14Q1, output_14Q2, output_14Q3, output_14Q4])

dt_df = dt.Frame(output_2014)
grouped = dt_df[:, [dt.sum(dt.f.C),
                  dt.sum(dt.f.I), dt.sum(dt.f.PS), dt.sum(dt.f.SS)],
                  dt.by(dt.f.drugname, #dt.f.prod_ai,
                  dt.f.year, dt.f.age_category,#dt.f.sex,
                  dt.f.indi_pt, dt.f.outc_cod)]

output_2014 = grouped.to_pandas()
output_2014.to_csv('output_2014.csv', index=False)


In [ ]:
output_2022 = pd.read_csv('output_2022.csv')
output_2023 = pd.read_csv('output_2023.csv')
output_2024 = pd.read_csv('output_2024.csv')

In [ ]:
consolidated_outputs =  pd.concat([output_2014, output_2015, output_2016, output_2017, output_2018,
                                   output_2019, output_2020, output_2021,
                                   output_2022, output_2023, output_2024])

dt_df = dt.Frame(consolidated_outputs)
grouped = dt_df[:, [dt.sum(dt.f.C),
                  dt.sum(dt.f.I), dt.sum(dt.f.PS), dt.sum(dt.f.SS)],
                  dt.by(dt.f.prod_ai,dt.f.age_category,
                  dt.f.year, dt.f.outc_cod)]

consolidated_outputs = grouped.to_pandas()
consolidated_outputs.to_csv('consolidated_outputs.csv', index=False)

In [ ]:
consolidated_outputs

,prod_ai,age_category,year,outc_cod,C,I,PS,SS
0,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...,Adolescent,2014,HO,0,0,2,2
1,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...,Adolescent,2015,HO,0,0,3,2
2,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...,Adolescent,2016,HO,0,0,4,0
3,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...,Adolescent,2016,OT,0,0,1,0
4,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...,Adolescent,2017,OT,1,0,2,0
...,...,...,...,...,...,...,...,...
643373,ZURANOLONE,Adult,2024,OT,0,0,12,0
643374,ZURANOLONE,Adult,2024,RI,0,0,3,0
643375,ZURANOLONE,Toddler,2023,LT,0,0,1,0
643376,ZURANOLONE,Toddler,2024,HO,0,0,4,0


In [ ]:
#get results from outc_data where primary_id and case_id are 65388687	6538868

#find the distinct values in outc_cod
temp_4['age_cod'].unique()

array(['YR', nan, 'WK', 'MON', 'DEC', 'DY', 'HR'], dtype=object)

In [ ]:
demo_data.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt',
       'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr',
       'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod',
       'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country'],
      dtype='object')

In [ ]:
#sort demo_data by primaryid and caseid and case version: reference: https://github.com/kylechua/faers-toolkit
demo_data = demo_data.sort_values(by=['caseid','caseversion'])

#keep only the latest case version
demo_data = demo_data.drop_duplicates(subset=['caseid'], keep='last')


In [ ]:
#start joining all the tables

#join demo_data and drug_data and primaryid and caseid
temp_1 = pd.merge(demo_data, drug_data, on=['primaryid','caseid'], how='left')

print(len(demo_data))#400,514

print(len(temp_1)) #2,008,162

400514
2008162


In [ ]:
#join temp1 with indi_data
temp_2 = pd.merge(temp_1, indi_data,  left_on=['primaryid','caseid','drug_seq'], right_on=['primaryid','caseid','indi_drug_seq'], how='left')

print(len(temp_2)) #2,023,181

2023181


In [ ]:
#join with outc_data
temp_3 = pd.merge(temp_2, outc_data, on=['primaryid','caseid'], how='left')

print(len(temp_3)) #3,714,690


3714690


In [ ]:
#before joining with other tables, convert the tables to primaryid and caseid level by concatenating the  pt column
reac_data_new = reac_data.groupby(['primaryid','caseid'])['pt'].apply('+'.join).reset_index()

#now join
temp_4 = pd.merge(temp_3, reac_data_new, on=['primaryid','caseid'], how='left')

print(len(temp_4)) #3,714,690


3714690


In [ ]:
#now join with rpsr_data
temp_5 = pd.merge(temp_4, rpsr_data, on=['primaryid','caseid'], how='left')

print(len(temp_5)) #3,715,384

3715384


In [ ]:
#now join with ther_data
temp_6 = pd.merge(temp_5, ther_data, left_on=['primaryid','caseid','drug_seq'],
                  right_on=['primaryid','caseid','dsg_drug_seq'],
                  how='left')

print(len(temp_6)) #3,798,054

3798054


In [ ]:
#delete all other tables; only keep temp_6
del demo_data
del drug_data
del indi_data
del outc_data
del reac_data
del reac_data_new
del rpsr_data
del ther_data
del temp_1
del temp_2
del temp_3
del temp_4
del temp_5

In [ ]:
#run missing value analysis on all columns of temp_6
temp_6.isnull().sum()

,0
primaryid,0
caseid,0
caseversion,0
i_f_code,0
event_dt,2489329
mfr_dt,0
init_fda_dt,0
fda_dt,0
rept_cod,0
auth_num,3551985


In [ ]:
#drop all columns with >50% missing values from temp_6
new_data = temp_6.dropna(thresh=0.5*len(temp_6), axis=1)

In [ ]:
###data processing

#consider init_fda_dt as the reporting data
#convert init_fda_dt into date column
new_data['init_fda_dt'] = pd.to_datetime(new_data['init_fda_dt'], format='%Y%m%d')

#drop mfr_dt, fda_dt, route, dose_vbm, dechal, mfr_num, occp_cod, val_vbm, rept_dt, occr_country
new_data = new_data.drop(['mfr_dt', 'fda_dt', 'route', 'dose_vbm','dechal','mfr_num','occp_cod','val_vbm','rept_dt','occr_country'], axis=1)

<ipython-input-24-c3073f92dfbd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['init_fda_dt'] = pd.to_datetime(new_data['init_fda_dt'], format='%Y%m%d')


In [ ]:
#keep only distinct values of new_data
new_data = new_data.drop_duplicates()

In [ ]:
len(new_data)

3624053

In [ ]:
#find distinct values in i_f_code
new_data['i_f_code'].unique()

array(['F', 'I'], dtype=object)

In [ ]:
new_data.head()

,primaryid,caseid,caseversion,i_f_code,init_fda_dt,rept_cod,mfr_sndr,age,age_cod,sex,e_sub,reporter_country,drug_seq,role_cod,drugname,prod_ai,indi_drug_seq,indi_pt,outc_cod,pt
0,62150432,6215043,2,F,2007-01-12,EXP,PFIZER,85.0,YR,F,Y,FR,1,PS,SOLU-MEDROL,METHYLPREDNISOLONE SODIUM SUCCINATE,NaN,NaN,HO,Supraventricular extrasystoles+Hypertension+Ac...
1,62150432,6215043,2,F,2007-01-12,EXP,PFIZER,85.0,YR,F,Y,FR,1,PS,SOLU-MEDROL,METHYLPREDNISOLONE SODIUM SUCCINATE,NaN,NaN,OT,Supraventricular extrasystoles+Hypertension+Ac...
2,62150432,6215043,2,F,2007-01-12,EXP,PFIZER,85.0,YR,F,Y,FR,2,SS,TAXOL,PACLITAXEL,2.0,Breast cancer female,HO,Supraventricular extrasystoles+Hypertension+Ac...
3,62150432,6215043,2,F,2007-01-12,EXP,PFIZER,85.0,YR,F,Y,FR,2,SS,TAXOL,PACLITAXEL,2.0,Breast cancer female,OT,Supraventricular extrasystoles+Hypertension+Ac...
4,62150432,6215043,2,F,2007-01-12,EXP,PFIZER,85.0,YR,F,Y,FR,3,SS,RANITIDINE HYDROCHLORIDE,RANITIDINE HYDROCHLORIDE,NaN,NaN,HO,Supraventricular extrasystoles+Hypertension+Ac...


In [ ]:
new_data.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'init_fda_dt',
       'rept_cod', 'mfr_sndr', 'age', 'age_cod', 'sex', 'e_sub',
       'reporter_country', 'occr_country', 'drug_seq', 'role_cod', 'drugname',
       'prod_ai', 'indi_drug_seq', 'indi_pt', 'outc_cod', 'pt'],
      dtype='object')

In [ ]:
#aggregate the data to the yearly level


In [ ]:
new_data.head(5)

,primaryid,caseid,caseversion,i_f_code,init_fda_dt,rept_cod,mfr_num,mfr_sndr,age,age_cod,...,drugname,prod_ai,val_vbm,route,dose_vbm,dechal,indi_drug_seq,indi_pt,outc_cod,pt
0,62150432,6215043,2,F,2007-01-12,EXP,FR-PFIZER INC-2007002279,PFIZER,85.0,YR,...,SOLU-MEDROL,METHYLPREDNISOLONE SODIUM SUCCINATE,1,Intravenous (not otherwise specified),"120 mg, 1x/day",D,NaN,NaN,HO,Supraventricular extrasystoles+Hypertension+Ac...
1,62150432,6215043,2,F,2007-01-12,EXP,FR-PFIZER INC-2007002279,PFIZER,85.0,YR,...,SOLU-MEDROL,METHYLPREDNISOLONE SODIUM SUCCINATE,1,Intravenous (not otherwise specified),"120 mg, 1x/day",D,NaN,NaN,OT,Supraventricular extrasystoles+Hypertension+Ac...
2,62150432,6215043,2,F,2007-01-12,EXP,FR-PFIZER INC-2007002279,PFIZER,85.0,YR,...,TAXOL,PACLITAXEL,1,Intravenous (not otherwise specified),"250 mg, 1x/day",D,2.0,Breast cancer female,HO,Supraventricular extrasystoles+Hypertension+Ac...
3,62150432,6215043,2,F,2007-01-12,EXP,FR-PFIZER INC-2007002279,PFIZER,85.0,YR,...,TAXOL,PACLITAXEL,1,Intravenous (not otherwise specified),"250 mg, 1x/day",D,2.0,Breast cancer female,OT,Supraventricular extrasystoles+Hypertension+Ac...
4,62150432,6215043,2,F,2007-01-12,EXP,FR-PFIZER INC-2007002279,PFIZER,85.0,YR,...,RANITIDINE HYDROCHLORIDE,RANITIDINE HYDROCHLORIDE,1,Intravenous (not otherwise specified),"50 mg, 1x/day",D,NaN,NaN,HO,Supraventricular extrasystoles+Hypertension+Ac...


In [ ]:
drug_data.head(5)

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100294532,10029453,1,PS,LETROZOLE,LETROZOLE,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,20726.0,NaN,NaN,NaN,NaN
1,100294532,10029453,2,SS,LAPATINIB,LAPATINIB,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100294532,10029453,3,SS,FULVESTRANT,FULVESTRANT,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100294532,10029453,4,SS,CAPECITABINE,CAPECITABINE,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100294532,10029453,5,SS,TRASTUZUMAB,TRASTUZUMAB,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
indi_data.head(5)

,primaryid,caseid,indi_drug_seq,indi_pt
0,100294532,10029453,1,Breast cancer metastatic
1,100294532,10029453,2,Breast cancer metastatic
2,100294532,10029453,3,Breast cancer metastatic
3,100294532,10029453,4,Breast cancer metastatic
4,100294532,10029453,5,Breast cancer metastatic


In [ ]:
outc_data.head(5)

,primaryid,caseid,outc_cod
0,100294532,10029453,OT
1,100496573,10049657,OT
2,1005762122,10057621,HO
3,100813192,10081319,CA
4,100813192,10081319,DE


In [ ]:
reac_data.head(5)

,primaryid,caseid,pt,drug_rec_act
0,100294532,10029453,Asthenia,NaN
1,100294532,10029453,Breast cancer metastatic,NaN
2,100294532,10029453,Palmar-plantar erythrodysaesthesia syndrome,NaN
3,100294532,10029453,Metastases to liver,NaN
4,100294532,10029453,Metastases to lymph nodes,NaN


In [ ]:
rpsr_data.head(5)

,primaryid,caseid,rpsr_cod
0,247995591,24799559,CSM
1,247995631,24799563,HP
2,247995741,24799574,HP
3,247995931,24799593,HP
4,247995971,24799597,HP


In [ ]:
rpsr_data.count()

,0
primaryid,11033
caseid,11033
rpsr_cod,11033


In [ ]:
ther_data.head(5)

,primaryid,caseid,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
0,100294532,10029453,2,200906.0,NaN,NaN,NaN
1,100294532,10029453,4,200901.0,NaN,NaN,NaN
2,100496573,10049657,1,20100330.0,NaN,NaN,NaN
3,100496573,10049657,2,201006.0,201202.0,14892.0,HR
4,100496573,10049657,3,20120511.0,NaN,NaN,NaN
